In [1]:
# Torch
import torch
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable

import time
import argparse
import os
import sys
# Our Modules
reader_folder = os.path.realpath(os.path.abspath('..'))
if reader_folder not in sys.path:
    sys.path.append(reader_folder)

import datasets
from datasets import utils
from models.MPNN import MPNN

from LogMetric import AverageMeter, Logger

from GraphReader.graph_reader import read_cxl


In [2]:
root = '../data/GREC/'
logPath = '../log/grec/mpnn/'
prefetch = 2
batch_size = 10
epochs = 360
lr = 0.001
lr_decay = 0.6
schedule = [0.1, 0.9]
log_interval = 100

In [3]:
train_classes, train_ids = read_cxl(os.path.join(root, 'data/train.cxl'))
test_classes, test_ids = read_cxl(os.path.join(root, 'data/test.cxl'))
valid_classes, valid_ids = read_cxl(os.path.join(root, 'data/valid.cxl'))

num_classes = len(list(set(train_classes + test_classes)))

data_train = datasets.GREC(root, train_ids, train_classes)
data_valid = datasets.GREC(root, valid_ids, valid_classes)
data_test = datasets.GREC(root, test_ids, test_classes)

# Define model and optimizer
print('Define model')
# Select one graph
g_tuple, l = data_train[0]
g, h_t, e = g_tuple

Define model


In [4]:
# Data Loader
train_loader = torch.utils.data.DataLoader(data_train,
                                           batch_size=batch_size, shuffle=True, collate_fn=datasets.utils.collate_g,
                                           num_workers=prefetch, pin_memory=False)
valid_loader = torch.utils.data.DataLoader(data_valid,
                                           batch_size=batch_size, collate_fn=datasets.utils.collate_g,
                                           num_workers=prefetch, pin_memory=False)
test_loader = torch.utils.data.DataLoader(data_test,
                                          batch_size=batch_size, collate_fn=datasets.utils.collate_g,
                                          num_workers=prefetch, pin_memory=False)

In [5]:
print('\tCreate model')
model = MPNN([len(h_t[0]), len(list(e.values())[0])], 25, 15, 2, num_classes, type='classification')

print('Optimizer')
optimizer = optim.Adam(model.parameters(), lr=lr)

criterion = nn.NLLLoss()

evaluation = utils.accuracy

print('Logger')
logger = Logger(logPath)

lr_step = (lr - lr * lr_decay)/(epochs * schedule[1] - epochs * schedule[0])


	Create model
Optimizer
Logger


In [6]:
def train(train_loader, model, criterion, optimizer, epoch, evaluation, logger):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    accuracies = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (g, h, e, target) in enumerate(train_loader):
        # Prepare input data
        target = torch.squeeze(target).type(torch.LongTensor)
        g, h, e, target = Variable(g), Variable(h), Variable(e), Variable(target)

        # Measure data loading time
        data_time.update(time.time() - end)

        def closure():
            optimizer.zero_grad()
            # Compute output
            output = model(g, h, e)
            train_loss = criterion(output, target)

            acc = Variable(evaluation(output.data, target.data, topk=(1,))[0])

            # Logs
            losses.update(train_loss.data[0], g.size(0))
            accuracies.update(acc.data[0], g.size(0))
            # compute gradient and do SGD step
            train_loss.backward()
            return train_loss

        optimizer.step(closure)

        # Measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % log_interval == 0 and i > 0:
            
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Accuracy {acc.val:.4f} ({acc.avg:.4f})'
                  .format(epoch, i, len(train_loader), batch_time=batch_time,
                          data_time=data_time, loss=losses, acc=accuracies))
                          
    logger.log_value('train_epoch_loss', losses.avg)
    logger.log_value('train_epoch_accuracy', accuracies.avg)

    print('Epoch: [{0}] Average Accuracy {acc.avg:.3f}; Average Loss {loss.avg:.3f}; Avg Time x Batch {b_time.avg:.3f}'
          .format(epoch, acc=accuracies, loss=losses, b_time=batch_time))


In [7]:
for epoch in range(0, epochs):

    if epoch > epochs * schedule[0] and epoch < epochs * schedule[1]:
        lr -= lr_step
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch, evaluation, logger)

    # evaluate on test set
    acc1 = validate(valid_loader, model, criterion, evaluation, logger)

    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)
    utils.save_checkpoint({'epoch': epoch + 1, 'state_dict': model.state_dict(), 'best_acc1': best_acc1,
                           'optimizer': optimizer.state_dict(), }, is_best=is_best, directory=args.resume)

    # Logger step
    logger.log_value('learning_rate', args.lr).step()

layer 0
layer 1
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])


/Users/fredericstgo/nmp_qc/models/MPNN.py:97: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  res = nn.LogSoftmax()(res)


layer 0
layer 1
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])
torch.Size([20, 25])
torch.Size([20])
torch.Size([20, 22])
layer 0
layer 1
torch.Size([23, 25])
torch.Size([23])
torch.Size([23, 22])
torch.Size([23, 25])
torch.Size([23])
torch.Size([23, 22])
torch.Size([23, 25])
torch.Size([23])
torch.Size([23, 22])
torch.Size([23, 25])
torch.Size([23])
torch.Size([23, 22])
torch.Size([23, 25])
torch.Size([23])
torch.Size([23, 22])
torch.Size([23, 25])
torch.Size([23])
torch.Size([23, 22])
torch.Size([23, 25])
tor

layer 0
layer 1
torch.Size([22, 25])
torch.Size([22])
torch.Size([22, 22])
torch.Size([22, 25])
torch.Size([22])
torch.Size([22, 22])
torch.Size([22, 25])
torch.Size([22])
torch.Size([22, 22])
torch.Size([22, 25])
torch.Size([22])
torch.Size([22, 22])
torch.Size([22, 25])
torch.Size([22])
torch.Size([22, 22])
torch.Size([22, 25])
torch.Size([22])
torch.Size([22, 22])
torch.Size([22, 25])
torch.Size([22])
torch.Size([22, 22])
torch.Size([22, 25])
torch.Size([22])
torch.Size([22, 22])
torch.Size([22, 25])
torch.Size([22])
torch.Size([22, 22])
torch.Size([22, 25])
torch.Size([22])
torch.Size([22, 22])
layer 0
layer 1
torch.Size([24, 25])
torch.Size([24])
torch.Size([24, 22])
torch.Size([24, 25])
torch.Size([24])
torch.Size([24, 22])
torch.Size([24, 25])
torch.Size([24])
torch.Size([24, 22])
torch.Size([24, 25])
torch.Size([24])
torch.Size([24, 22])
torch.Size([24, 25])
torch.Size([24])
torch.Size([24, 22])
torch.Size([24, 25])
torch.Size([24])
torch.Size([24, 22])
torch.Size([24, 25])
tor

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/usr/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/usr/local/Cellar/python@2/2.7.14_3/Frameworks/Python.framework/Versions/2.7/lib/python2.7/inspect.py", line 1051, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/usr/local/Cellar/python@2/2.7.14_3/Frameworks/Python.framework/Versions/2.7/lib/python2.7/inspect.py", line 1011, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/usr/local/Cellar/python@2/2.7.14_3/Frameworks/Python.framework/Versio

IndexError: string index out of range